In [165]:

!pip install requests
# Connecting with CCF netowrks.
server="https://127.0.0.1:8000"

#User Ceriftiate creation and Registartion 

In [166]:
import requests
import json
import os

# Replace with your actual API endpoint URL
url = server + "/app/api/metrics"

try:
    response = requests.get(url, verify='service_cert.pem')

    print("Status Code:", response.status_code)
    print("\nResponse Headers:")
    for header, value in response.headers.items():
        print(f"{header}: {value}")

    print("\nResponse Body:")
    try:
        # Attempt to parse JSON and print it in an indented format
        response_json = response.json()
        print(json.dumps(response_json, indent=4))
    except ValueError:
        # If response is not JSON, print as plain text
        print(response.text)

except requests.exceptions.RequestException as e:
    print("Error making request:", e)


Status Code: 200

Response Headers:
content-length: 3820
content-type: application/json
x-ms-ccf-transaction-id: 2.98

Response Body:
{
    "metrics": [
        {
            "calls": 5,
            "errors": 0,
            "failures": 0,
            "method": "GET",
            "path": "/",
            "retries": 0
        },
        {
            "calls": 1,
            "errors": 0,
            "failures": 0,
            "method": "HEAD",
            "path": "/",
            "retries": 0
        },
        {
            "calls": 3,
            "errors": 0,
            "failures": 0,
            "method": "PUT",
            "path": "account/0f5190e0857a3340b1bcc773aa332b6f3c70a47c1ad77ac5c7d98463a36f275c/current_account",
            "retries": 0
        },
        {
            "calls": 1,
            "errors": 0,
            "failures": 0,
            "method": "PUT",
            "path": "account/0f5190e0857a3340b1bcc773aa332b6f3c70a47c1ad77ac5c7d98463a36f275c/savings_account",
    

In [168]:
# Keygenrator function for public and private key with ballat request file
import subprocess
import sys
DEFAULT_CURVE = "secp384r1"
FAST_CURVE = "secp256r1"
SUPPORTED_CURVES = [DEFAULT_CURVE, FAST_CURVE]

DIGEST_SHA384 = "sha384"
DIGEST_SHA256 = "sha256"

RSA_SIZE = 2048

def generate_keys(name, curve=DEFAULT_CURVE, generate_encryption_key=False):
    if not name:
        print("Error: The name of the participant should be specified (e.g. member0 or user1)")
        sys.exit(1)

    if curve not in SUPPORTED_CURVES:
        print(f"{curve} curve is not in {SUPPORTED_CURVES}")
        sys.exit(1)

    digest = DIGEST_SHA384 if curve == DEFAULT_CURVE else DIGEST_SHA256

    cert = f"{name}_cert.pem"
    privk = f"{name}_privk.pem"

    print(f"-- Generating identity private key and certificate for participant \"{name}\"...")
    print(f"Identity curve: {curve}")

    subprocess.run(["openssl", "ecparam", "-out", privk, "-name", curve, "-genkey"], check=True)
    subprocess.run(["openssl", "req", "-new", "-key", privk, "-x509", "-nodes", "-days", "365", "-out", cert, f"-{digest}", "-subj", f"/CN={name}"], check=True)

    print(f"Identity private key generated at: {privk}")
    print(f"Identity certificate generated at: {cert} (to be registered in CCF)")

    if generate_encryption_key:
        print(f"-- Generating RSA encryption key pair for participant \"{name}\"...")

        enc_priv = f"{name}_enc_privk.pem"
        enc_pub = f"{name}_enc_pubk.pem"

        subprocess.run(["openssl", "genrsa", "-out", enc_priv, str(RSA_SIZE)], check=True)
        subprocess.run(["openssl", "rsa", "-in", enc_priv, "-pubout", "-out", enc_pub], check=True)

        print(f"Encryption private key generated at: {enc_priv}")
        print(f"Encryption public key generated at: {enc_pub} (to be registered in CCF)")


In [169]:
# CA certificate creator function
def create_certificate(cert_name):
    cert_file = f"{cert_name}_cert.pem"
    set_user_file = f"set_{cert_name}.json"
    
    # Call the generate_keys function (make sure to include it in your script)
    generate_keys(cert_name)

    # Read the certificate file and format it
    with open(cert_file, 'r') as file:
        cert_content = file.read().replace('\n', '\n')

    # Create the JSON content
    user_json = {
        "actions": [
            {
                "name": "set_user",
                "args": {
                    "cert": cert_content
                }
            }
        ]
    }

    # Write the JSON to a file
    with open(set_user_file, 'w') as file:
        json.dump(user_json, file, indent=2)
    print(f"JSON file created at: {set_user_file}")
# Create user0 certificate
create_certificate("user4")

-- Generating identity private key and certificate for participant "user4"...
Identity curve: secp384r1
Identity private key generated at: user4_privk.pem
Identity certificate generated at: user4_cert.pem (to be registered in CCF)
JSON file created at: set_user4.json


In [170]:
import os
import ssl                                        
openssl_dir, openssl_cafile = os.path.split(      
    ssl.get_default_verify_paths().openssl_cafile)
# no content in this folder
os.listdir(openssl_dir)
# non existent file
print(os.path.exists(os.path.join(openssl_dir, openssl_cafile)))

True


In [171]:

import base64
import hashlib
from urllib.parse import urlparse
def send_secure_request(url, request_data_path, signing_privk, signing_cert, command="post"):
    def read_request_data(request_path):
        if request_path.startswith('@'):
            with open(request_path[1:], 'r') as file:
                return file.read()
        else:
            return request_path

    def calculate_digest(data):
        sha256_hash = hashlib.sha256()
        sha256_hash.update(data.encode('utf-8'))
        return base64.b64encode(sha256_hash.digest()).decode()

    def create_signature(string_to_sign, priv_key_path):
        process = subprocess.Popen(
            ['openssl', 'dgst', '-sha384', '-sign', priv_key_path],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        signature, _ = process.communicate(string_to_sign.encode())
        return base64.b64encode(signature).decode().replace('\n', '')

    def prepare_string_to_sign(method, url, digest, data_length):
        parsed_url = urlparse(url)
        path = parsed_url.path
        return f"(request-target): {method.lower()} {path}\ndigest: SHA-256={digest}\ncontent-length: {data_length}"

    def get_cert_key_id(cert_path):
        proc = subprocess.Popen(
            ['openssl', 'x509', '-in', cert_path, '-noout', '-fingerprint', '-sha256'],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        output, _ = proc.communicate()
        fingerprint = output.decode().split('=')[1].replace(':', '').lower().replace('\n', '')
        return fingerprint

    request_data = read_request_data(request_data_path)
    request_digest = calculate_digest(request_data)
    string_to_sign = prepare_string_to_sign(command, url, request_digest, str(len(request_data)))
    signature = create_signature(string_to_sign, signing_privk)
    key_id = get_cert_key_id(signing_cert)

    headers = {
        "Digest": f"SHA-256={request_digest}",
        "Authorization": f'Signature keyId="{key_id}",algorithm="hs2019",headers="(request-target) digest content-length",signature="{signature}"'
    }

    response = requests.post(url, data=request_data, headers=headers, verify='service_cert.pem')
    return response.status_code, response.text

# Example Usage
url = "https://127.0.0.1:8000/gov/proposals"
request_data_path = "@set_user4.json"
signing_privk = "member0_privk.pem"
signing_cert = "member0_cert.pem"

status_code, text = send_secure_request(url, request_data_path, signing_privk, signing_cert)
print(status_code, text)


200 {"ballot_count":0,"proposal_id":"02d5156597d279c89f93452d56336644674cce5625a9c3832264edc7bab1cdd7","proposer_id":"63eaef8bfb05cb485e9e1e3813a8ec68c73b99387bbd252b4c6dc7ee386b0e6c","state":"Open"}


In [172]:
import subprocess
import base64
import hashlib
from urllib.parse import urlparse

# sending voting to add  the user based on proposal id 

def send_ballot_request(server_url, proposal_id, data_file_path, signing_privk_path, signing_cert_path):
    def read_request_data(file_path):
        with open(file_path, 'r') as file:
            return file.read()

    def calculate_digest(data):
        sha256_hash = hashlib.sha256()
        sha256_hash.update(data.encode('utf-8'))
        return base64.b64encode(sha256_hash.digest()).decode()

    def create_signature(string_to_sign, priv_key_path):
        process = subprocess.Popen(
            ['openssl', 'dgst', '-sha384', '-sign', priv_key_path],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        signature, _ = process.communicate(string_to_sign.encode())
        return base64.b64encode(signature).decode().replace('\n', '')

    def prepare_string_to_sign(url, digest, data_length):
        parsed_url = urlparse(url)
        path = parsed_url.path
        return f"(request-target): post {path}\ndigest: SHA-256={digest}\ncontent-length: {data_length}"

    def get_cert_key_id(cert_path):
        proc = subprocess.Popen(
            ['openssl', 'x509', '-in', cert_path, '-noout', '-fingerprint', '-sha256'],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        output, _ = proc.communicate()
        fingerprint = output.decode().split('=')[1].replace(':', '').lower().replace('\n', '')
        return fingerprint

    url = f"{server_url}/gov/proposals/{proposal_id}/ballots"
    request_data = read_request_data(data_file_path)
    request_digest = calculate_digest(request_data)
    string_to_sign = prepare_string_to_sign(url, request_digest, str(len(request_data)))
    signature = create_signature(string_to_sign, signing_privk_path)
    key_id = get_cert_key_id(signing_cert_path)

    headers = {
        "Digest": f"SHA-256={request_digest}",
        "Authorization": f'Signature keyId="{key_id}",algorithm="hs2019",headers="(request-target) digest content-length",signature="{signature}"',
        "Content-Type": "application/json"
    }

    response = requests.post(url, data=request_data, headers=headers, verify='service_cert.pem')
    return response.status_code, response.json()

# Example Usage
server_url = "https://127.0.0.1:8000"
proposal_id = "02d5156597d279c89f93452d56336644674cce5625a9c3832264edc7bab1cdd7"  # Replace with the actual proposal ID
data_file_path = "vote_accept.json"
signing_privk_path = "member1_privk.pem"
signing_cert_path = "member1_cert.pem"

status_code, response_json = send_ballot_request(server_url, proposal_id, data_file_path, signing_privk_path, signing_cert_path)
print(status_code, response_json)


200 {'ballot_count': 1, 'proposal_id': '02d5156597d279c89f93452d56336644674cce5625a9c3832264edc7bab1cdd7', 'proposer_id': '63eaef8bfb05cb485e9e1e3813a8ec68c73b99387bbd252b4c6dc7ee386b0e6c', 'state': 'Open'}


In [173]:
server_url = "https://127.0.0.1:8000"
proposal_id = "02d5156597d279c89f93452d56336644674cce5625a9c3832264edc7bab1cdd7"  # Replace with the actual proposal ID
data_file_path = "vote_accept.json"
signing_privk_path = "member2_privk.pem"
signing_cert_path = "member2_cert.pem"

status_code, response_json = send_ballot_request(server_url, proposal_id, data_file_path, signing_privk_path, signing_cert_path)
print(status_code, response_json)

200 {'ballot_count': 2, 'proposal_id': '02d5156597d279c89f93452d56336644674cce5625a9c3832264edc7bab1cdd7', 'proposer_id': '63eaef8bfb05cb485e9e1e3813a8ec68c73b99387bbd252b4c6dc7ee386b0e6c', 'state': 'Accepted', 'votes': {'62e18acdb30fbf8bb9cc6ec15daf7746ded6b5ca04f38b9138147b3c8e649543': True, '99a17c7b5d953f660ca377004ea5ac699da0cb42ea34bae5a9af7d2d0a3ff9ae': True}}


In [174]:
import requests

def get_ccf_version(server_url, cert_path):
    url = f"{server_url}/node/version"
    response = requests.get(url, verify=cert_path)
    return response.status_code, response.json()

def get_network_details(server_url, cert_path):
    url = f"{server_url}/node/network"
    response = requests.get(url, verify=cert_path)
    return response.status_code, response.json()

def get_network_nodes_details(server_url, cert_path):
    url = f"{server_url}/node/network/nodes"
    response = requests.get(url, verify=cert_path)
    return response.status_code, response.json()

def get_network_members_details(server_url, cert_path):
    url = f"{server_url}/gov/members"
    response = requests.get(url, verify=cert_path)
    return response.status_code, response.json()

# Example Usage
server_url = "https://127.0.0.1:8000"  # Replace with your actual server URL
cert_path = "service_cert.pem"  # Replace with your actual cert path

version_status, version_info = get_ccf_version(server_url, cert_path)
print("CCF Version:", version_status, version_info)

network_status, network_info = get_network_details(server_url, cert_path)
print("Network Details:", network_status, network_info)

nodes_status, nodes_info = get_network_nodes_details(server_url, cert_path)
print("Network Nodes Details:", nodes_status, nodes_info)

members_status, members_info = get_network_members_details(server_url, cert_path)
print


CCF Version: 200 {'ccf_version': 'ccf-3.0.12', 'quickjs_version': '2021-03-27', 'unsafe': False}
Network Details: 200 {'current_service_create_txid': '2.1', 'current_view': 2, 'primary_id': 'd545cef0cd5f4362d3cd75e7bef63f3b2f6eed518a876d12f0874a984b9748da', 'recovery_count': 0, 'service_certificate': '-----BEGIN CERTIFICATE-----\nMIIBvjCCAUOgAwIBAgIQTRZOJMa6oFczpRYy/nxM1jAKBggqhkjOPQQDAzAWMRQw\nEgYDVQQDDAtDQ0YgTmV0d29yazAeFw0yMzExMTExNjIwMTdaFw0yNDAyMDkxNjIw\nMTZaMBYxFDASBgNVBAMMC0NDRiBOZXR3b3JrMHYwEAYHKoZIzj0CAQYFK4EEACID\nYgAEQz75QF8cA+anmanKOm0kGxulMDD2qxGKjM47Y0Jx9HT3mYYBzuyTbBWsYKBH\nOe+xXVZBsglNd7CAwn0tIZPtRQORxIGU3SgE0Dr1LsJyJ3/NQwOoiTREfbbGY37q\nN1xwo1YwVDASBgNVHRMBAf8ECDAGAQH/AgEAMB0GA1UdDgQWBBRvd6WisCGVREA6\nFSCPMqM1aS2u8jAfBgNVHSMEGDAWgBRvd6WisCGVREA6FSCPMqM1aS2u8jAKBggq\nhkjOPQQDAwNpADBmAjEAo+dO4Yx7fkFa1utwMU8oBGqxcMSqqq/sF8Z3HphWe9v6\nzfnA9hrKtwEG3BaDWpKUAjEAl/vNYZiNW4kb1E7QS3q0hUtjSbwuvOdQ02ZpGczu\nSjiHX53s3HVFodCisZfRv7mG\n-----END CERTIFICATE-----\n', 'service_data': No

<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [175]:
import subprocess

# creation of user id based on certifacte 
def get_certificate_fingerprint(cert_path):
    process = subprocess.Popen(
        ['openssl', 'x509', '-in', cert_path, '-noout', '-fingerprint', '-sha256'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    output, _ = process.communicate()
    fingerprint = output.decode().split('=')[1].replace(':', '').lower().strip()
    return fingerprint

# user0_id = get_certificate_fingerprint("user3_cert.pem")
# user1_id = get_certificate_fingerprint("user3_cert.pem")
user3_id = get_certificate_fingerprint("user4_cert.pem")
account_type0 = 'current_account'
account_type1 = 'savings_account'

# print("User 0 ID:", user0_id)
# print("User 1 ID:", user1_id)
print("User 1 ID:", user3_id)
print("Account Type 0:", account_type0)
print("Account Type 1:", account_type1)


User 1 ID: f486811a0606fcd06ffc537bf729653cacf9fee1fd27bcbc33886d8f16c66386
Account Type 0: current_account
Account Type 1: savings_account


In [231]:
import requests

def create_account_for_user(server_url, user_id, account_type, cert_path, key_path, cacert_path):
    url = f"{server_url}/app/account/{user_id}/{account_type}"
    response = requests.put(url, cert=(cert_path, key_path), verify=cacert_path)
    # Check if the response is JSON
    try:
        response_json = response.json()
    except ValueError:  # Includes simplejson.decoder.JSONDecodeError
        response_json = response.text  # Fallback to raw text

    return response.status_code, response_json

# Example Usage
server = "https://127.0.0.1:8000"  # Replace with your actual server URL
user0_id = "008a99260f0eedb29cd45668a5a9d111cf818c90d40535ad0b8fd0372f60b96b"  # Replace with the actual user0_id
account_type0 = 'current_account'  # Replace with the actual account type
member0_cert = "member0_cert.pem"  # Replace with the path to your member0 certificate
member0_privk = "member0_privk.pem"  # Replace with the path to your member0 private key
service_cert = "service_cert.pem"  # Replace with the path to your CA certificate

status_code, response_json = create_account_for_user(server, user0_id, account_type0, member0_cert, member0_privk, service_cert)
print(status_code, response_json)


204 


In [230]:
import requests

def check_user_balance(server_url, account_type, user_cert_path, user_key_path, cacert_path):
    url = f"{server_url}/app/balance/{account_type}"
    response = requests.get(url, cert=(user_cert_path, user_key_path), verify=cacert_path)
    return response.status_code, response.json()

# Example Usage
server = "https://127.0.0.1:8000"  # Replace with your actual server URL
account_type0 = 'current_account'  # Replace with the actual account type
user0_cert = "user4_cert.pem"  # Replace with the path to your user0 certificate
user0_privk = "user4_privk.pem"  # Replace with the path to your user0 private key
service_cert = "service_cert.pem"  # Replace with the path to your CA certificate

status_code, balance_info = check_user_balance(server, account_type0, user0_cert, user0_privk, service_cert)
print("Status Code:", status_code)
print("Balance Info:", balance_info)


OSError: Could not find the TLS certificate file, invalid path: user4_cert.pem

In [183]:
import requests

def deposit_to_account(server_url, user_id, account_type, amount, cert_path, key_path, cacert_path):
    url = f"{server_url}/app/deposit/{user_id}/{account_type}"
    headers = {'Content-Type': 'application/json'}
    payload = {"value": amount}
    
    response = requests.post(url, json=payload, headers=headers, cert=(cert_path, key_path), verify=cacert_path)
    
    # Handle non-JSON responses
    try:
        response_data = response.json()
    except ValueError:
        response_data = response.text

    return response.status_code, response_data

# Example Usage
server = "https://127.0.0.1:8000"
user0_id = "f486811a0606fcd06ffc537bf729653cacf9fee1fd27bcbc33886d8f16c66386"
account_type0 = 'current_account'
deposit_amount = 100
member0_cert = "member0_cert.pem"
member0_privk = "member0_privk.pem"
service_cert = "service_cert.pem"

status_code, response_data = deposit_to_account(server, user0_id, account_type0, deposit_amount, member0_cert, member0_privk, service_cert)
print("Status Code:", status_code)
print("Response Data:", response_data)


Status Code: 204
Response Data: 


In [184]:
import requests

def check_user_balance(server_url, account_type, user_cert_path, user_key_path, cacert_path):
    url = f"{server_url}/app/balance/{account_type}"
    response = requests.get(url, cert=(user_cert_path, user_key_path), verify=cacert_path)
    return response.status_code, response.json()

# Example Usage
server = "https://127.0.0.1:8000"  # Replace with your actual server URL
account_type0 = 'current_account'  # Replace with the actual account type
user0_cert = "user4_cert.pem"  # Replace with the path to your user0 certificate
user0_privk = "user4_privk.pem"  # Replace with the path to your user0 private key
service_cert = "service_cert.pem"  # Replace with the path to your CA certificate

status_code, balance_info = check_user_balance(server, account_type0, user0_cert, user0_privk, service_cert)
print("Status Code:", status_code)
print("Balance Info:", balance_info)

Status Code: 200
Balance Info: {'balance': 100}


In [188]:
import requests

def transfer_funds(server_url, from_account_type, to_account_type, amount, user_from_cert, user_from_key, cacert_path, user_to_id):
    url = f"{server_url}/app/transfer/{from_account_type}"
    headers = {'Content-Type': 'application/json'}
    payload = {
        "value": amount,
        "user_id_to": user_to_id,
        "account_name_to": to_account_type
    }
    
    response = requests.post(url, json=payload, headers=headers, cert=(user_from_cert, user_from_key), verify=cacert_path)

    # Extracting transaction ID from headers
    transaction_id = response.headers.get('X-MS-CCF-Transaction-Id', '').strip()

    # Check if the response is JSON
    try:
        response_json = response.json()
    except ValueError:  # Includes simplejson.decoder.JSONDecodeError
        response_json = response.text  # Fallback to raw text

    return transaction_id, response.status_code, response_json

# Example Usage
server = "https://127.0.0.1:8000"  # Replace with your server URL
account_type0 = 'current_account'  # User0's account type
account_type1 = 'current_account'  # User1's account type
amount = 40  # Amount to transfer
user0_cert = "user1_cert.pem"  # Path to User0's certificate
user0_privk = "user1_privk.pem"  # Path to User0's private key
service_cert = "service_cert.pem"  # Path to the CA certificate
user1_id = "f486811a0606fcd06ffc537bf729653cacf9fee1fd27bcbc33886d8f16c66386"  # Replace with User1's ID

transfer_transaction_id, status_code, response_json = transfer_funds(server, account_type0, account_type1, amount, user0_cert, user0_privk, service_cert, user1_id)
print("Transaction ID:", transfer_transaction_id)
print("Status Code:", status_code)
print("Response:", response_json)


Transaction ID: 
Status Code: 401
Response: {'error': {'code': 'InvalidAuthenticationInfo', 'details': [{'auth_policy': 'BaseAuthPolicy', 'code': 'InvalidAuthenticationInfo', 'message': 'Could not find matching user certificate'}], 'message': 'Invalid info'}}


In [ ]:
import time
import requests

def wait_for_receipt(server_url, transaction_id, user_cert, user_key, cacert_path):
    url = f"{server_url}/app/receipt?transaction_id={transaction_id}"
    while True:
        response = requests.get(url, cert=(user_cert, user_key), verify=cacert_path)
        print("response",response)
        if response.status_code == 200:
            break
        time.sleep(1000)

# Example Usage
server = "https://127.0.0.1:8000"  # Replace with your server URL
transfer_transaction_id = "2.57"  # Replace with the actual transaction ID
user0_cert = "user0_cert.pem"  # Path to User0's certificate
user0_privk = "user0_privk.pem"  # Path to User0's private key
service_cert = "service_cert.pem"  # Path to the CA certificate

wait_for_receipt(server, transfer_transaction_id, user0_cert, user0_privk, service_cert)


In [189]:
import requests

def check_users_balances(server_url, users, cacert_path):
    balances = {}
    for user in users:
        account_type = user['account_type']
        user_cert = user['cert']
        user_key = user['key']
        user_id = user['id']

        url = f"{server_url}/app/balance/{account_type}"
        response = requests.get(url, cert=(user_cert, user_key), verify=cacert_path)
        
        balance_key = f"{user_id} - {account_type}"
        if response.status_code == 200:
            balances[balance_key] = response.json()
        else:
            balances[balance_key] = f"Error: {response.status_code}"

    return balances

# Example Usage
server = "https://127.0.0.1:8000"
service_cert = "service_cert.pem"

users = [
    {"id": "user0", "account_type": 'current_account', "cert": "user0_cert.pem", "key": "user0_privk.pem"},
    {"id": "user0", "account_type": 'savings_account', "cert": "user0_cert.pem", "key": "user0_privk.pem"},
    {"id": "user1", "account_type": 'current_account', "cert": "user1_cert.pem", "key": "user1_privk.pem"},
    {"id": "user1", "account_type": 'savings_account', "cert": "user1_cert.pem", "key": "user1_privk.pem"},
    {"id": "user3", "account_type": 'savings_account', "cert": "user3_cert.pem", "key": "user3_privk.pem"},
    {"id": "user3", "account_type": 'current_account', "cert": "user3_cert.pem", "key": "user3_privk.pem"},
      {"id": "user4", "account_type": 'savings_account', "cert": "user4_cert.pem", "key": "user4_privk.pem"},
    {"id": "user4", "account_type": 'current_account', "cert": "user4_cert.pem", "key": "user4_privk.pem"}
]

balances = check_users_balances(server, users, service_cert)
for account_key, balance_info in balances.items():
    print(f"Balance for {account_key}: {balance_info}")


Balance for user0 - current_account: Error: 401
Balance for user0 - savings_account: Error: 401
Balance for user1 - current_account: Error: 401
Balance for user1 - savings_account: Error: 401
Balance for user3 - savings_account: Error: 401
Balance for user3 - current_account: Error: 401
Balance for user4 - savings_account: Error: 404
Balance for user4 - current_account: {'balance': 100}


In [136]:
import subprocess
import json
import requests
from urllib.parse import urlparse

# Constants for the script
server_url = "https://127.0.0.1:8000"
DEFAULT_CURVE = "secp384r1"
DIGEST_SHA384 = "sha384"
RSA_SIZE = 2048

# Function to generate keys and certificates
def generate_keys(name, curve=DEFAULT_CURVE):
    if not name:
        print("Error: The name of the participant should be specified (e.g., member0 or user1)")
        sys.exit(1)

    if curve not in [DEFAULT_CURVE, "secp256r1"]:
        print(f"{curve} curve is not supported")
        sys.exit(1)

    digest = DIGEST_SHA384 if curve == DEFAULT_CURVE else "sha256"
    cert = f"{name}_cert.pem"
    privk = f"{name}_privk.pem"

    subprocess.run(["openssl", "ecparam", "-out", privk, "-name", curve, "-genkey"], check=True)
    subprocess.run(["openssl", "req", "-new", "-key", privk, "-x509", "-nodes", "-days", "365", "-out", cert, f"-{digest}", "-subj", f"/CN={name}"], check=True)

    print(f"Identity private key generated at: {privk}")
    print(f"Identity certificate generated at: {cert} (to be registered in CCF)")

# Function to extract user ID from a certificate
def get_certificate_fingerprint(cert_path):
    process = subprocess.Popen(
        ['openssl', 'x509', '-in', cert_path, '-noout', '-fingerprint', '-sha256'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    output, _ = process.communicate()
    fingerprint = output.decode().split('=')[1].replace(':', '').lower().strip()
    return fingerprint

# Function to create a certificate and JSON proposal
def create_certificate(cert_name):
    cert_file = f"{cert_name}_cert.pem"
    set_user_file = f"set_{cert_name}.json"
    generate_keys(cert_name)

    with open(cert_file, 'r') as file:
        cert_content = file.read().replace('\n', '\\n')

    user_json = {
        "actions": [{"name": "set_user", "args": {"cert": cert_content}}]
    }

    with open(set_user_file, 'w') as file:
        json.dump(user_json, file, indent=2)

# Function to send a secure request
def send_secure_request(url, request_data_path, signing_privk, signing_cert, command="post"):
    # Include the read_request_data, calculate_digest, create_signature, prepare_string_to_sign, get_cert_key_id functions here

    request_data = read_request_data(request_data_path)
    request_digest = calculate_digest(request_data)
    string_to_sign = prepare_string_to_sign(command, url, request_digest, str(len(request_data)))
    signature = create_signature(string_to_sign, signing_privk)
    key_id = get_cert_key_id(signing_cert)

    headers = {
        "Digest": f"SHA-256={request_digest}",
        "Authorization": f'Signature keyId="{key_id}",algorithm="hs2019",headers="(request-target) digest content-length",signature="{signature}"'
    }

    response = requests.post(url, data=request_data, headers=headers, verify='service_cert.pem')
    return response.status_code, response.text

# Main process to register users and print their details
def register_and_print_users(num_users):
    for i in range(num_users):
        user_name = f"user{i}"
        create_certificate(user_name)
        user_id = get_certificate_fingerprint(f"{user_name}_cert.pem")
        print(f"User {i} ID:", user_id)
        print("Account Type 0: current_account")
        print("Account Type 1: savings_account")

# Call the main function with the desired number of users
num_users = 5
register_and_print_users(num_users)
def submit_add_user_proposal(user_cert_path, member_cert_path, member_key_path):
    user_id = get_certificate_fingerprint(user_cert_path)
    proposal = {
        "actions": [
            {
                "name": "set_user",
                "args": {"cert": user_id}
            }
        ]
    }
    url = f"{server_url}/gov/proposals"
    response = requests.post(url, json=proposal, cert=(member_cert_path, member_key_path), verify='service_cert.pem')
    proposal_id = response.json().get("proposal_id")
    return proposal_id, response.status_code

# Function to vote on a proposal
def vote_on_proposal(proposal_id, vote, member_cert_path, member_key_path):
    url = f"{server_url}/gov/proposals/{proposal_id}/ballots"
    response = requests.post(url, json={"ballot": vote}, cert=(member_cert_path, member_key_path), verify='service_cert.pem')
    return response.status_code

# Main function to register users
def register_users(num_users):
    for i in range(num_users):
        user_name = f"user{i}"
        user_cert_path = f"{user_name}_cert.pem"
        create_certificate(user_name)

        # Submit proposal to add user
        proposal_id, _ = submit_add_user_proposal(user_cert_path, "member0_cert.pem", "member0_privk.pem")

        # Each member votes on the proposal
        for j in range(3):  # Assuming 3 members for voting
            member_cert = f"member{j}_cert.pem"
            member_key = f"member{j}_privk.pem"
            vote_on_proposal(proposal_id, "accept", member_cert, member_key)

        print(f"User {i} registered with ID: {get_certificate_fingerprint(user_cert_path)}")

# Run the script for a specified number of users

Identity private key generated at: user0_privk.pem
Identity certificate generated at: user0_cert.pem (to be registered in CCF)
User 0 ID: 216225c41ce1c93b66ab2e68ff3fc04a9cc8fd62a526d952e2e6a1e29c83d44f
Account Type 0: current_account
Account Type 1: savings_account
Identity private key generated at: user1_privk.pem
Identity certificate generated at: user1_cert.pem (to be registered in CCF)
User 1 ID: 7af3b7a9311583fbd561cd1b33c8f83ceb4e6da7457d1d7ac4ee466a07a34da0
Account Type 0: current_account
Account Type 1: savings_account
Identity private key generated at: user2_privk.pem
Identity certificate generated at: user2_cert.pem (to be registered in CCF)
User 2 ID: 12600eaf8ae9da0c98e1fd5cf36b4982609081c087f0c3dfe1429b90c526d61c
Account Type 0: current_account
Account Type 1: savings_account
Identity private key generated at: user3_privk.pem
Identity certificate generated at: user3_cert.pem (to be registered in CCF)
User 3 ID: 66fbf03ac80d477babb0e78cfce80db944014933455227bb6b637f30fb056

In [196]:
import subprocess
import json
import requests
import time
from urllib.parse import urlparse

# Constants and Configuration
server_url = "https://127.0.0.1:8000"
DEFAULT_CURVE = "secp384r1"
DIGEST_SHA384 = "sha384"
RSA_SIZE = 2048
service_cert = "service_cert.pem"

# Key Generator Function
def generate_keys(name, curve=DEFAULT_CURVE, generate_encryption_key=False):
    if not name:
        print("Error: The name of the participant should be specified (e.g. member0 or user1)")
        sys.exit(1)

    if curve not in [DEFAULT_CURVE, "secp256r1"]:
        print(f"{curve} curve is not in SUPPORTED_CURVES")
        sys.exit(1)

    digest = DIGEST_SHA384 if curve == DEFAULT_CURVE else "sha256"

    cert = f"{name}_cert.pem"
    privk = f"{name}_privk.pem"

    print(f"-- Generating identity private key and certificate for participant \"{name}\"...")
    print(f"Identity curve: {curve}")

    subprocess.run(["openssl", "ecparam", "-out", privk, "-name", curve, "-genkey"], check=True)
    subprocess.run(["openssl", "req", "-new", "-key", privk, "-x509", "-nodes", "-days", "365", "-out", cert, f"-{digest}", "-subj", f"/CN={name}"], check=True)

    print(f"Identity private key generated at: {privk}")
    print(f"Identity certificate generated at: {cert} (to be registered in CCF)")

    if generate_encryption_key:
        print(f"-- Generating RSA encryption key pair for participant \"{name}\"...")

        enc_priv = f"{name}_enc_privk.pem"
        enc_pub = f"{name}_enc_pubk.pem"

        subprocess.run(["openssl", "genrsa", "-out", enc_priv, str(RSA_SIZE)], check=True)
        subprocess.run(["openssl", "rsa", "-in", enc_priv, "-pubout", "-out", enc_pub], check=True)

        print(f"Encryption private key generated at: {enc_priv}")
        print(f"Encryption public key generated at: {enc_pub} (to be registered in CCF)")

# Certificate Creator Function
def create_certificate(cert_name):
    cert_file = f"{cert_name}_cert.pem"
    set_user_file = f"set_{cert_name}.json"
    
    generate_keys(cert_name)

    with open(cert_file, 'r') as file:
        cert_content = file.read().replace('\n', '\n')

    user_json = {
        "actions": [
            {
                "name": "set_user",
                "args": {
                    "cert": cert_content
                }
            }
        ]
    }

    with open(set_user_file, 'w') as file:
        json.dump(user_json, file, indent=2)
    print(f"JSON file created at: {set_user_file}")

# Secure Request Sender Function
def send_secure_request(url, request_data_path, signing_privk, signing_cert, command="post"):
    def read_request_data(request_path):
        if request_path.startswith('@'):
            with open(request_path[1:], 'r') as file:
                return file.read()
        else:
            return request_path

    def calculate_digest(data):
        sha256_hash = hashlib.sha256()
        sha256_hash.update(data.encode('utf-8'))
        return base64.b64encode(sha256_hash.digest()).decode()

    def create_signature(string_to_sign, priv_key_path):
        process = subprocess.Popen(
            ['openssl', 'dgst', '-sha384', '-sign', priv_key_path],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        signature, _ = process.communicate(string_to_sign.encode())
        return base64.b64encode(signature).decode().replace('\n', '')

    def prepare_string_to_sign(method, url, digest, data_length):
        parsed_url = urlparse(url)
        path = parsed_url.path
        return f"(request-target): {method.lower()} {path}\ndigest: SHA-256={digest}\ncontent-length: {data_length}"

    def get_cert_key_id(cert_path):
        proc = subprocess.Popen(
            ['openssl', 'x509', '-in', cert_path, '-noout', '-fingerprint', '-sha256'],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        output, _ = proc.communicate()
        fingerprint = output.decode().split('=')[1].replace(':', '').lower().replace('\n', '')
        return fingerprint

    request_data = read_request_data(request_data_path)
    request_digest = calculate_digest(request_data)
    string_to_sign = prepare_string_to_sign(command, url, request_digest, str(len(request_data)))
    signature = create_signature(string_to_sign, signing_privk)
    key_id = get_cert_key_id(signing_cert)

    headers = {
        "Digest": f"SHA-256={request_digest}",
        "Authorization": f'Signature keyId="{key_id}",algorithm="hs2019",headers="(request-target) digest content-length",signature="{signature}"'
    }

    response = requests.post(url, data=request_data, headers=headers, verify='service_cert.pem')
    return response.status_code, response.text




In [250]:
def main_process(num_initial_users):
    # Generate keys and certificates for all users
    for i in range(num_initial_users):
        create_certificate(f"user{i}")

    # Submit proposal and voting for each user
    for i in range(num_initial_users):
        user_name = f"user{i}"
        user_cert_path = f"{user_name}_cert.pem"
        user_key_path = f"{user_name}_privk.pem"

        # Member0 submits the proposal
        status_code, response_json = send_secure_request(server_url + "/gov/proposals", f"@set_{user_name}.json", "member0_privk.pem", "member0_cert.pem", "post")
        print("proposal_id",json.loads(response_json)['proposal_id'])
        proposal_id=json.loads(response_json)['proposal_id']

        # Other members vote
        for j in range(1, 3):  # Assuming 2 other members (member1 and member2)
            vote_status, response_json = send_ballot_request(server_url, proposal_id, "vote_accept.json", f"member{j}_privk.pem", f"member{j}_cert.pem")
            # print("vote_status",vote_status,"response_json",response_json)
            if vote_status != 200:
                print(f"Voting failed for {user_name} by member{j}")
                continue
            time.sleep(1)  # Delay for processing

        # Create account for the user
        user_id=get_certificate_fingerprint(user_cert_path)
        print("Actual id",user_id) 
        account_status,_= create_account_for_user(server_url,user_id, 'saving_account', "member0_cert.pem", "member0_privk.pem", service_cert)
        if account_status != 204:
            print(f"Account creation failed for {user_name}")
            continue
        else:
            print("Account created succesuflly")

        # Deposit funds
      
        deposit_status, _ = deposit_to_account(server_url,user_id, 'saving_account', 100, "member0_cert.pem", "member0_privk.pem", service_cert)
        print("print statu",deposit_status)
        
        if deposit_status != 204:
            print(f"Deposit failed for {user_name}")
            continue
        else:
            print("Deposite sucessfully")

        # Check balance
        balance_status, balance_info = check_user_balance(server_url, 'saving_account', user_cert_path, user_key_path, service_cert)
        print("balance_info",balance_info,balance_status)
        if balance_status != 200:
            print(f"Failed to check balance for {user_name}")
            continue

        print(f"User {i} (ID: {user_name}) - Balance after deposit: {balance_info}")

# Run the script for a specified number of users
main_process(10)

-- Generating identity private key and certificate for participant "user0"...
Identity curve: secp384r1
Identity private key generated at: user0_privk.pem
Identity certificate generated at: user0_cert.pem (to be registered in CCF)
JSON file created at: set_user0.json
-- Generating identity private key and certificate for participant "user1"...
Identity curve: secp384r1
Identity private key generated at: user1_privk.pem
Identity certificate generated at: user1_cert.pem (to be registered in CCF)
JSON file created at: set_user1.json
-- Generating identity private key and certificate for participant "user2"...
Identity curve: secp384r1
Identity private key generated at: user2_privk.pem
Identity certificate generated at: user2_cert.pem (to be registered in CCF)
JSON file created at: set_user2.json
-- Generating identity private key and certificate for participant "user3"...
Identity curve: secp384r1
Identity private key generated at: user3_privk.pem
Identity certificate generated at: user3_

In [251]:

import os
import glob

def delete_user_files(num_users):
    for i in range(num_users):
        # File names based on user index
        cert_file = f"user{i}_cert.pem"
        priv_key_file = f"user{i}_privk.pem"
        enc_priv_file = f"user{i}_enc_privk.pem"
        enc_pub_file = f"user{i}_enc_pubk.pem"

        # Delete the files if they exist
        for file in [cert_file, priv_key_file, enc_priv_file, enc_pub_file]:
            if os.path.exists(file):
                os.remove(file)
                print(f"Deleted {file}")
            else:
                print(f"{file} does not exist")

        # Delete all JSON files related to the user
        for json_file in glob.glob(f"set_user{i}*.json"):
            os.remove(json_file)
            print(f"Deleted {json_file}")

# Specify the number of users to delete
num_users = 10
delete_user_files(num_users)


Deleted user0_cert.pem
Deleted user0_privk.pem
user0_enc_privk.pem does not exist
user0_enc_pubk.pem does not exist
Deleted set_user0.json
Deleted user1_cert.pem
Deleted user1_privk.pem
user1_enc_privk.pem does not exist
user1_enc_pubk.pem does not exist
Deleted set_user1.json
Deleted user2_cert.pem
Deleted user2_privk.pem
user2_enc_privk.pem does not exist
user2_enc_pubk.pem does not exist
Deleted set_user2.json
Deleted user3_cert.pem
Deleted user3_privk.pem
user3_enc_privk.pem does not exist
user3_enc_pubk.pem does not exist
Deleted set_user3.json
Deleted user4_cert.pem
Deleted user4_privk.pem
user4_enc_privk.pem does not exist
user4_enc_pubk.pem does not exist
Deleted set_user4.json
Deleted user5_cert.pem
Deleted user5_privk.pem
user5_enc_privk.pem does not exist
user5_enc_pubk.pem does not exist
Deleted set_user5.json
Deleted user6_cert.pem
Deleted user6_privk.pem
user6_enc_privk.pem does not exist
user6_enc_pubk.pem does not exist
Deleted set_user6.json
Deleted user7_cert.pem
Dele